# Création d'un model LSTM

##  Importation des librairies 

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2024-04-02 22:42:02.160121: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Récupération des données 

In [2]:
chemin_fichier_csv = 'loto.csv'

df = pd.read_csv(chemin_fichier_csv, sep=';') 

print(df.columns)

Index(['annee_numero_de_tirage', 'date_de_tirage', 'boule_1', 'boule_2',
       'boule_3', 'boule_4', 'boule_5', 'numero_chance'],
      dtype='object')


## Quelques statistiques sur la base de données

In [3]:
print(df.head())
print(df.info())
print(df.isnull().sum())
print(df.describe())

   annee_numero_de_tirage date_de_tirage  boule_1  boule_2  boule_3  boule_4  \
0                20244040     2024-04-01        9       17       15       12   
1                20244039     2024-03-30       40       41       28        8   
2                20244038     2024-03-27       28       45        2       15   
3                20244037     2024-03-25       37        1       22       26   
4                20244036     2024-03-23       47       49       20       34   

   boule_5  numero_chance  
0       26              8  
1       42              2  
2       23              7  
3       31              5  
4       32              1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2425 entries, 0 to 2424
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   annee_numero_de_tirage  2425 non-null   int64 
 1   date_de_tirage          2425 non-null   object
 2   boule_1                 2425 non

## Convertir les colonnes de dates

In [4]:
df['date_de_tirage'] = pd.to_datetime(df['date_de_tirage'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2425 entries, 0 to 2424
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   annee_numero_de_tirage  2425 non-null   int64         
 1   date_de_tirage          2425 non-null   datetime64[ns]
 2   boule_1                 2425 non-null   int64         
 3   boule_2                 2425 non-null   int64         
 4   boule_3                 2425 non-null   int64         
 5   boule_4                 2425 non-null   int64         
 6   boule_5                 2425 non-null   int64         
 7   numero_chance           2425 non-null   int64         
dtypes: datetime64[ns](1), int64(7)
memory usage: 151.7 KB
None


## Nettoyage des colonnes numériques

In [5]:
num_columns = ['boule_1', 'boule_2', 'boule_3', 'boule_4', 'boule_5', 'numero_chance']

for col in num_columns:
    df[col] = df[col].astype(int)

print(df[num_columns].dtypes)

boule_1          int64
boule_2          int64
boule_3          int64
boule_4          int64
boule_5          int64
numero_chance    int64
dtype: object


## Traitement des valeurs manquantes

In [6]:
# Pour la colonne 'numero_chance', par exemple
df['numero_chance'] = df['numero_chance'].fillna(df['numero_chance'].median())

# Supprimer les lignes où au moins un élément est manquant
df.dropna(inplace=True)

## Préparation des données

In [7]:
# Supposons que `df` est votre DataFrame Pandas contenant les tirages du loto
# Convertissons cela en un tableau numpy pour simplifier l'exemple
data = df[['boule_1', 'boule_2', 'boule_3', 'boule_4', 'boule_5', 'numero_chance']].to_numpy()

# Création des séquences (exemple simple)
X = []
y = []
for i in range(len(data) - 1):
    X.append(data[i])
    y.append(data[i + 1])

X = np.array(X)
y = np.array(y)

# Pour cet exemple, nous n'appliquons pas de catégorisation one-hot sur `y` 
# car cela dépendrait du nombre maximal possible pour chaque boule/numéro.


## Modélisation

In [8]:
# Nombre de caractéristiques (le nombre de boules par tirage)
n_features = X.shape[1]

# Définition du modèle LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(1, n_features)),
    Dense(n_features)
])

model.compile(optimizer='adam', loss='mse')


## Formation du modèle

In [9]:
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Entraînement du modèle
model.fit(X, y, epochs=200, verbose=0)


## Évaluation du modèle

In [10]:
# Génération d'une prédiction
predicted = model.predict(X[-1].reshape((1, 1, n_features)))
print(predicted)


1/1 [==============================] - 0s 187ms/step
[[30.201904 23.7052   25.47495  28.7322   26.10588   5.785817]]
